In [3]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import LinearRegression

In [ ]:
df_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet') # for training 
df_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet') # for validation and test

In [1]:
# df = pd.concat([df_jan,df_feb],axis=0)
df = df_jan

NameError: name 'df_jan' is not defined

In [6]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


In [7]:
len(df.columns)

19

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5972150 entries, 0 to 3007525
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee            floa

In [9]:
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds()/60

In [10]:
df.describe()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
count,5.972150e+06,5972150,5972150,5.646378e+06,5.972150e+06,5.646378e+06,5.972150e+06,5.972150e+06,5.972150e+06,5.972150e+06,5.972150e+06,5.972150e+06,5.972150e+06,5.972150e+06,5.972150e+06,5.972150e+06,5.646378e+06,5.646378e+06,5.972150e+06
mean,1.754723e+00,2024-02-01 00:27:21.513338,2024-02-01 00:43:09.454327,1.332613e+00,3.757264e+00,2.092253e+00,1.657988e+02,1.648061e+02,1.148489e+00,1.811072e+01,1.448330e+00,4.833992e-01,3.319238e+00,5.172432e-01,9.754978e-01,2.671232e+01,2.261295e+00,1.351430e-01,1.579902e+01
min,1.000000e+00,2002-12-31 22:59:39,2002-12-31 23:05:41,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,-9.990000e+02,-7.500000e+00,-5.000000e-01,-1.400000e+02,-8.000000e+01,-1.000000e+00,-1.000000e+03,-2.500000e+00,-1.750000e+00,-5.206667e+01
25%,2.000000e+00,2024-01-17 12:38:32.250000,2024-01-17 12:56:59,1.000000e+00,1.000000e+00,1.000000e+00,1.320000e+02,1.140000e+02,1.000000e+00,8.600000e+00,0.000000e+00,5.000000e-01,1.000000e+00,0.000000e+00,1.000000e+00,1.540000e+01,2.500000e+00,0.000000e+00,7.250000e+00
50%,2.000000e+00,2024-02-01 10:13:11,2024-02-01 10:30:28,1.000000e+00,1.690000e+00,1.000000e+00,1.620000e+02,1.620000e+02,1.000000e+00,1.280000e+01,1.000000e+00,5.000000e-01,2.700000e+00,0.000000e+00,1.000000e+00,2.016000e+01,2.500000e+00,0.000000e+00,1.181667e+01
75%,2.000000e+00,2024-02-15 19:25:02,2024-02-15 19:40:37,1.000000e+00,3.100000e+00,1.000000e+00,2.340000e+02,2.340000e+02,1.000000e+00,2.050000e+01,2.500000e+00,5.000000e-01,4.120000e+00,0.000000e+00,1.000000e+00,2.856000e+01,2.500000e+00,0.000000e+00,1.895000e+01
max,6.000000e+00,2024-03-01 00:01:37,2024-03-01 23:24:42,9.000000e+00,3.127223e+05,9.900000e+01,2.650000e+02,2.650000e+02,4.000000e+00,9.792000e+03,1.425000e+01,5.750000e+00,5.433300e+02,1.159200e+02,1.000000e+00,9.792000e+03,2.500000e+00,1.750000e+00,9.455400e+03
std,4.312623e-01,NaN,NaN,8.414196e-01,2.406166e+02,9.946784e+00,6.379422e+01,6.930429e+01,5.858684e-01,1.879168e+01,1.800763e+00,1.177540e-01,3.862271e+00,2.104544e+00,2.187255e-01,2.317853e+01,8.183577e-01,4.783123e-01,3.456991e+01


In [11]:
len(df['duration'][(df['duration']>=1) & (df['duration']<60)])/len(df['duration'])

0.977344842309721

In [12]:
categorical = ['PULocationID', 'DOLocationID']

df[categorical] = df[categorical].astype('str')

In [13]:
train_data = df[categorical].to_dict(orient='records')

In [14]:
dv = DictVectorizer()

In [15]:
X_train = dv.fit_transform(train_data)

: 